In [ ]:
import pandas as pd
import numpy as np
import math

#read the Spectramax data
print('Enter the name of the data file')
x=input()
df=pd.read_excel(x+'.xlsx')
df.columns=['Sample','Wells','Pathlength','A260','A280','A320','A230','A260/280']
df=df.drop(df.index[0:2])
df=df.reset_index()
df=df.drop('index', axis=1)

#read COA files

coa=input('Enter the name of the COA file')
df5=pd.read_excel(coa+'.xls')

#set dataframe
df2_temp=pd.DataFrame()
df2=pd.DataFrame()

#user enter plate names and transform into list
input_string = input("Enter the plates separated by space, example:60 61 62 63 64 65 ")
plate  = input_string.split()

#grab no. of plate from COA
for i in range(4):
    df2_temp=df5.loc[df5['Plate Name'] == 'DSPNGSRNA_'+str(plate[i])]
    df2=df2.append(df2_temp)

#reset index
df2=df2.reset_index()
df2=df2.drop('index', axis=1)

#getting first plate
firstplate1=df.loc[0:15:2]
firstplate2=df.loc[32:47:2]
firstplate3=df.loc[64:79:2]
firstplate4=df.loc[96:111:2]
firstplate5=df.loc[128:143:2]
firstplate6=df.loc[160:175:2]
firstplate7=df.loc[192:207:2]
firstplate8=df.loc[224:239:2]
firstplate9=df.loc[256:271:2]
firstplate10=df.loc[288:303:2]
firstplate11=df.loc[320:335:2]
firstplate12=df.loc[352:367:2]

firstplate=[firstplate1,firstplate2,firstplate3,firstplate4,firstplate5,firstplate6,firstplate7,firstplate8,firstplate9,firstplate10,firstplate11,firstplate12]
firstplate=pd.concat(firstplate)
firstplate=firstplate.reset_index()
firstplate=firstplate.drop('index',axis=1)

#add ext coef into firstplate
ext=df2.loc[:,'Ext Coeff']
 
firstplate.loc[:,'Ext_Coef']=ext.loc[0:95]
#print(firstplate.loc[:,'Ext_Coef'])

#add well position for 96 plates
platewell=df2.loc[0:95,'Well Position']
firstplate['Wells_96']=platewell
#print(firstplate)

#create the concentration and ND8000 column and add into firstplate
firstplate['Dilution']=200
firstplate.loc[:,'Concentration_SPM']=firstplate.A260*firstplate.Dilution*1000000/firstplate.Ext_Coef
firstplate.loc[:,'ND8000']=firstplate.A260*firstplate.Dilution*0.85*1000000/firstplate.Ext_Coef

#add conc from europhins
conc=df2.loc[:,'Conc']
firstplate['Conc_Euro']=conc.loc[0:95]

#add plate name into first plate
platename1=df2.loc[:,'Plate Name']
firstplate.loc[:,'plate_name']=platename1

#getting second plate
secondplate1=df.loc[16:31:2]
secondplate2=df.loc[48:63:2]
secondplate3=df.loc[80:95:2]
secondplate4=df.loc[112:127:2]
secondplate5=df.loc[144:159:2]
secondplate6=df.loc[176:191:2]
secondplate7=df.loc[208:223:2]
secondplate8=df.loc[240:255:2]
secondplate9=df.loc[272:287:2]
secondplate10=df.loc[304:319:2]
secondplate11=df.loc[336:351:2]
secondplate12=df.loc[368:383:2]

secondplate=[secondplate1,secondplate2,secondplate3,secondplate4,secondplate5,secondplate6,secondplate7,secondplate8,secondplate9,secondplate10,secondplate11,secondplate12]
secondplate=pd.concat(secondplate)
secondplate=secondplate.reset_index()
secondplate=secondplate.drop('index',axis=1)

#add ext coef into secondplate
ext2=ext.loc[96:191]
ext2=ext2.reset_index()
ext2=ext2.drop('index',axis=1)
secondplate['Ext_Coef']=ext2

#add well 96
secondplate['Wells_96']=platewell

#create the ND8000 column and add into secondplate
secondplate['Dilution']=200
secondplate.loc[:,'Concentration_SPM']=secondplate.A260*secondplate.Dilution*1000000/secondplate.Ext_Coef
secondplate.loc[:,'ND8000']=secondplate.A260*secondplate.Dilution*0.85*1000000/secondplate.Ext_Coef

#add conc euro
conc2=conc.loc[96:191]
conc2=conc2.reset_index()
conc2=conc2.drop('index',axis=1)
secondplate['Conc_Euro']=conc2


#add plate name into second plate
platename2=platename1.loc[96:191]
platename2=platename2.reset_index()
platename2=platename2.drop('index',axis=1)
secondplate['plate_name']=platename2


#combining all plate and export to excel
finaldata=[firstplate,secondplate]
finaldata=pd.concat(finaldata)
finaldata=finaldata.reset_index()
finaldata=finaldata.drop('index',axis=1)

#add tgt conc and tgt vol and Oligo Vol and TE Vol
finaldata['Tgt_Conc']=20.0
finaldata['Tgt_Vol']=60.0
finaldata['Oligo_Vol']= np.where(finaldata['ND8000']>finaldata['Tgt_Conc'],finaldata['Tgt_Conc']*finaldata['Tgt_Vol']/finaldata['ND8000'],finaldata['Tgt_Vol'])
finaldata['TE_Vol']=np.where(finaldata['ND8000']>finaldata['Tgt_Conc'],finaldata['ND8000']*finaldata['Oligo_Vol']/finaldata['Tgt_Conc']-finaldata['Oligo_Vol'],0)

finaldata=finaldata[['Wells','Wells_96','plate_name','Pathlength','A260','A280','A320','A230','A260/280','Dilution','Ext_Coef','Concentration_SPM','ND8000','Conc_Euro','Tgt_Conc','Tgt_Vol','Oligo_Vol','TE_Vol']]

finaldata.to_excel(x+'_Transform.xlsx')
print('The output excel file is name ' + x +'_Transform.xlsx')
